## 1

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My\ Drive/self\ driving/data/driving_dataset

/content/drive/My Drive/self driving/data/driving_dataset


In [0]:
# Credits: https://github.com/SullyChen/Autopilot-TensorFlow
# Research paper: End to End Learning for Self-Driving Cars by Nvidia. [https://arxiv.org/pdf/1604.07316.pdf]

# NVidia dataset: 72 hrs of video => 72*60*60*30 = 7,776,000 images
# Nvidia blog: https://devblogs.nvidia.com/deep-learning-self-driving-cars/


# Our Dataset: https://github.com/SullyChen/Autopilot-TensorFlow [https://drive.google.com/file/d/0B-KJCaaF7elleG1RbzVPZWV4Tlk/view]
# Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB


# If you want to try on a slightly large dataset: 70 minutes of data ~ 223GB
# Refer: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5
# Format: Image, latitude, longitude, gear, brake, throttle, steering angles and speed



# Additional Installations:
# pip3 install h5py


# AWS: https://aws.amazon.com/blogs/machine-learning/get-started-with-deep-learning-using-the-aws-deep-learning-ami/

# Youtube:https://www.youtube.com/watch?v=qhUvQiKec2U
# Further reading and extensions: https://medium.com/udacity/teaching-a-machine-to-steer-a-car-d73217f2492c
# More data: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5

In [0]:
from google.colab.patches import cv2_imshow
import os
import tensorflow as tf
#import scipy
#from __future__ import division
import gzip
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error


import os
import numpy as np
import random
import pickle
#from scipy import pi
#from itertools import islice
#import tensorflow as tf
#import scipy.misc
import cv2
from keras.models import load_model
from subprocess import call
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers,callbacks
#import keras
from keras import Sequential
from keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D,BatchNormalization
from keras.initializers import TruncatedNormal as tcd
from keras.regularizers import l2,l1

Using TensorFlow backend.


In [0]:
import gc
gc.collect()

83

## pre prcss

In [0]:
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("data.txt") as f:
    for line in f:
        xs.append(line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * np.pi / 180)

#get number of images
num_images = len(xs)

In [0]:
train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

In [0]:
num_train_images = len(train_xs)
num_val_images = len(val_xs)
num_train_images,num_val_images

(31784, 13621)

In [0]:
flow = pd.DataFrame({'path':train_xs,'target':train_ys})

flow_test = pd.DataFrame({'path':val_xs,'target':val_ys})

In [0]:
import gzip
def train_data(file,np):
  if np==2 or np==3:
    return None
  trn=[]
  for i in range(len(file)):
    x=cv2.imread(file[i])[-150:]
    x=cv2.resize(x,(66,200))
    x=x/255.0
    trn.append(x)
    if i%500==0: print("process ",np,', ',i," files read")
  file=gzip.open('trn/train_data_'+str(np),'wb')
  pickle.dump(trn,file)
  file.close()
  return None

In [0]:
%%time
pool = mp.Pool()
no_prcss= mp.cpu_count()

nn = int(len(train_xs)/no_prcss)
res=[]
for i in [0,1]:
  res.append(pool.apply_async(train_data,args=(flow.path.iloc[i*nn:(i+1)*nn].values,i)))
pool.close()
pool.join()

In [0]:
import gzip
def test_data(file,np):
  trn=[]
  for i in range(len(file)):
    x=cv2.imread(file[i])[-150:]
    x=cv2.resize(x,(66,200))
    x=x/255.0
    trn.append(x)
    if i%500==0: print("process ",np,', ',i," files read")
  file=gzip.open('tst/test_data_'+str(np),'wb')
  pickle.dump(trn,file)
  file.close()
  return None

In [0]:
%%time
pool = mp.Pool()
no_prcss= 2 #mp.cpu_count()

nn = int(len(val_xs)/no_prcss)
res=[]
for i in range(no_prcss):
  res.append(pool.apply_async(test_data,args=(flow_test.path.iloc[i*nn:(i+1)*nn].values,i)))
pool.close()
pool.join()

In [0]:
trn=[]
for i in tqdm(range(4)):
  f = gzip.open('/content/drive/My Drive/self driving/data/train_data/train_data_'+str(i),'rb')
  trn.extend(pickle.load(f))
  f.close()
tst=[]
for i in tqdm(range(2)):
  f = gzip.open('/content/drive/My Drive/self driving/data/test_data/test_data_'+str(i),'rb')
  tst.extend(pickle.load(f))
  f.close()

## modeling

In [0]:
trn=[]
for i in tqdm(range(4)):
  f = gzip.open('/content/drive/My Drive/self driving/data/train_data/train_data_'+str(i),'rb')
  trn.extend(pickle.load(f))
  f.close()
tst=[]
for i in tqdm(range(2)):
  f = gzip.open('/content/drive/My Drive/self driving/data/test_data/test_data_'+str(i),'rb')
  tst.extend(pickle.load(f))
  f.close()

In [0]:
trn = np.array(trn)

In [0]:
tst = np.array(tst)

In [0]:
model = Sequential()
model.add(Conv2D(64, (5,5), input_shape=(200, 66, 3) , data_format='channels_last',activation='relu', kernel_initializer='he_normal')) #, kernel_initializer=tcd(stddev=0.1)
#model.add(Dropout(rate=0.5))
model.add(MaxPooling2D(2,2))
#model.add(Conv2D(36,5,5,activation='relu'))
#model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,5,5,activation='relu',kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
#model.add(Dropout(rate=0.3))
model.add(MaxPooling2D(2,2))

#model.add(Conv2D(64,3,3,activation='relu'))
model.add(Conv2D(64,3,3,activation='relu'))
#model.add(Dropout(rate=0.3))
#model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,1,1,activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Flatten())

model.add(Dropout(rate=0.5))
model.add(Dense(1164,activation='relu')) #,kernel_regularizer=l1(0.005)
#model.add(Dropout(rate=0.4))

model.add(Dense(100,activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(rate=0.4))

model.add(Dense(50,activation='relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(10,activation='sigmoid'))

model.add(Dense(1,activation='elu'))
model.compile(optimizers.Adam(), loss='mean_squared_error')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_128 (Conv2D)          (None, 196, 62, 64)       4864      
_________________________________________________________________
max_pooling2d_95 (MaxPooling (None, 98, 31, 64)        0         
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 94, 27, 128)       204928    
_________________________________________________________________
max_pooling2d_96 (MaxPooling (None, 47, 13, 128)       0         
_________________________________________________________________
conv2d_130 (Conv2D)          (None, 45, 11, 64)        73792     
_________________________________________________________________
conv2d_131 (Conv2D)          (None, 45, 11, 64)        4160      
_________________________________________________________________
max_pooling2d_97 (MaxPooling (None, 22, 5, 64)         0         
__________

In [0]:
cllbk = callbacks.ModelCheckpoint('modelf',monitor='val_loss',save_best_only=True,mode='min',verbose=0)
h=model.fit(trn[int(len(trn)/2):],train_ys[int(len(trn)/2):],batch_size=128,epochs=15,validation_data=(tst,val_ys[:-1]),callbacks=[cllbk])

Train on 15892 samples, validate on 13620 samples
Epoch 1/15
15892/15892 [==============================] - 28s 2ms/step - loss: 0.1545 - val_loss: 0.1801
Epoch 2/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0673 - val_loss: 0.1941
Epoch 3/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0448 - val_loss: 0.1493
Epoch 4/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0357 - val_loss: 0.1285
Epoch 5/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0308 - val_loss: 0.1225
Epoch 6/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0278 - val_loss: 0.1213
Epoch 7/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0270 - val_loss: 0.1295
Epoch 8/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0246 - val_loss: 0.1206
Epoch 9/15
15892/15892 [==============================] - 22s 1ms/step - loss: 0.0225 - val_loss: 0.1242
Epoch

In [0]:
model = load_model('modelf')
ypred=model.predict(tst,verbose=1)

13620/13620 [==============================] - 9s 634us/step


In [0]:
img = cv2.imread('steering_wheel_image.jpg',1)
img=cv2.resize(img, (455,256))
rows,cols,d = img.shape

smoothed_angle = 0

#cap = cv2.VideoCapture(0)
out = cv2.VideoWriter('output_vdo.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (455,512)) #('M','J','P','G')

i = 0 
image = cv2.imread(val_xs[i], 1)
#with open('outfile.txt', 'w') as outfile
outfile = open('outfile.txt', 'w')

for i in tqdm(range(len(ypred)-1)) :    
    full_image = cv2.imread(val_xs[i], 1)    
    degrees = ypred[i][0] * 180.0 / np.pi
    outfile.write("{}. Steering angle: {} (pred), {} (actual) \n".format(i,str(np.round(degrees,5)),str(np.round(val_ys[i]*180/np.pi,3))))    #cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    vis = np.concatenate((full_image, dst), axis=0)
    out.write(vis)
    
#cv2.destroyAllWindows()
out.release()
outfile.close()

## Conclusion:

In [0]:
print("Mean square error on Test data: ",mean_squared_error(val_ys[:-1],ypred))

Mean square error: 0.11452212034143198
